In [1]:
suppressPackageStartupMessages({
    suppressWarnings({
        library(data.table)
        library(tidyverse)
        library(tidymodels)
    })
})

In [52]:
set.seed(1)

spl <- initial_split(iris,prop = .7)

train_set <- spl %>% training

spl_ <- initial_split(train_set,prop = .7)

train_fr <- spl_ %>% training

test_fr <- spl_ %>% testing

gorme <- spl %>% testing

In [55]:
tarif_xgb <- recipe(formula = Sepal.Width ~ .,data = train_fr) %>%
step_rm(Species)

spc_xgb <- boost_tree() %>% 
set_mode('regression') %>%
set_engine('xgboost')

wf_xgb <- workflow() %>%
add_model(spc_xgb) %>%
add_recipe(tarif_xgb)

In [56]:
tarif_lm <- recipe(formula = Sepal.Width ~ .,data = train_fr) %>%
step_rm(Species)

spc_lm <- linear_reg() %>% 
set_mode('regression') %>%
set_engine('lm')

wf_lm <- workflow() %>%
add_model(spc_lm) %>%
add_recipe(tarif_lm)

In [91]:
tarif_randfor <- recipe(formula = Sepal.Width ~ .,data = train_fr) %>%
step_rm(Species)

spc_randfor <- rand_forest() %>% 
set_mode('regression') %>%
set_engine('ranger')

wf_randfor <- workflow() %>%
add_model(spc_randfor) %>%
add_recipe(tarif_randfor)

In [92]:
wf_xgb_ <- wf_xgb %>% fit(train_fr)

wf_lm_ <- wf_lm %>% fit(train_fr)

wf_randfor_ <- wf_randfor %>% fit(train_fr)

In [101]:
trained_wfs <- list(wf_lm_,wf_randfor_)

In [107]:
stacked_model <- function(trained_workflows,test_set,dep_var,target_frame){
    
    stack_frame <- data.frame(drop_this = 1:nrow(test_set))
    
    target_preds <- data.frame(drop_this = 1:nrow(target_frame))
    
    for(i in 1:length(trained_workflows)){
        predictions <- pull(predict(trained_workflows[[i]],test_set))
        
        target_predictions <- pull(predict(trained_workflows[[i]],target_frame))
        
        model_name <- trained_workflows[[i]]$fit$actions$model$spec$engine
        
        column_pred <- data.frame(predictions)
        
        column_target <- data.frame(target_predictions)
        
        names(column_pred) <- model_name
        
        names(column_target) <- model_name
        
        stack_frame <- bind_cols(stack_frame,column_pred)
        
        target_preds <- bind_cols(target_preds,column_target)
    }
    
    stack_frame <- bind_cols(stack_frame,dep_var = test_set[[dep_var]]) %>% select(-drop_this)
    
    target_predictions <- target_preds %>% select(-drop_this)
    
    stacked_model <- lm(dep_var ~ . - 1,stack_frame) 
    #stacked_model
    target_predictions %>% mutate(pred = predict(stacked_model,.))
}

In [108]:
aa <- stacked_model(trained_wfs,test_set = test_fr,dep_var = 'Sepal.Width',target_frame = gorme) 

In [109]:
aa

lm,ranger,pred
<dbl>,<dbl>,<dbl>
3.231563,3.133195,3.150454
3.041193,3.221079,3.173787
3.373954,3.168593,3.210551
3.312709,3.171093,3.198294
2.966681,3.101027,3.064486
3.584224,3.583189,3.577131
3.898113,3.723224,3.757155
3.361716,3.284213,3.296378
3.047827,3.168949,3.135349


In [110]:
aa


lm,ranger,pred
<dbl>,<dbl>,<dbl>
3.231563,3.133195,3.150454
3.041193,3.221079,3.173787
3.373954,3.168593,3.210551
3.312709,3.171093,3.198294
2.966681,3.101027,3.064486
3.584224,3.583189,3.577131
3.898113,3.723224,3.757155
3.361716,3.284213,3.296378
3.047827,3.168949,3.135349


In [111]:
gorme %>% bind_cols(aa) %>%
summarise(rmse_randfor = rmse_vec(truth = Sepal.Width,estimate = ranger),
                    rmse_lm = rmse_vec(truth = Sepal.Width,estimate = lm),
                    rmse_stacked = rmse_vec(truth = Sepal.Width,estimate = pred))

rmse_randfor,rmse_lm,rmse_stacked
<dbl>,<dbl>,<dbl>
0.2770467,0.3061507,0.269475
